### Based on
https://towardsdatascience.com/a-beginners-guide-to-building-a-retrieval-augmented-generation-rag-application-from-scratch-e52921953a5d

### Note
Make sure to run:

$ ollama serve

to get it serving on the localhost before running this notebook.

In [143]:
import requests
import json

In [144]:
corpus_of_documents = [
    "Take a leisurely walk in the park and enjoy the fresh air.",
    "Visit a local museum and discover something new.",
    "Attend a live music concert and feel the rhythm.",
    "Go for a hike and admire the natural scenery.",
    "Have a picnic with friends and share some laughs.",
    "Explore a new cuisine by dining at an ethnic restaurant.",
    "Take a yoga class and stretch your body and mind.",
    "Join a local sports league and enjoy some friendly competition.",
    "Attend a workshop or lecture on a topic you're interested in.",
    "Visit an amusement park and ride the roller coasters."
]

In [145]:
def jaccard_similarity(query, document):
    query = query.lower().split(" ")
    document = document.lower().split(" ")
    intersection = set(query).intersection(set(document))
    union = set(query).union(set(document))
    return len(intersection)/len(union)

In [146]:
def return_response(query, corpus):
    similarities = []
    for doc in corpus:
        similarity = jaccard_similarity(query, doc)
        similarities.append(similarity)
    return corpus_of_documents[similarities.index(max(similarities))]

In [147]:
# user_prompt = "What is a leisure activity that you like?"
user_input = "I like to hike"
relevant_document = return_response(user_input, corpus_of_documents)
full_response = []

In [148]:
prompt = f"""
You are a bot that makes recommendations for activities. You answer in short sentences. Do not include extra information.
This is the recommended activity: {relevant_document}
The user input is: {user_input}
Compile a recommendation to the user based on the recommended activity and the user input.
"""
prompt

'\nYou are a bot that makes recommendations for activities. You answer in short sentences. Do not include extra information.\nThis is the recommended activity: Go for a hike and admire the natural scenery.\nThe user input is: I like to hike\nCompile a recommendation to the user based on the recommended activity and the user input.\n'

In [149]:
url = 'http://localhost:11434/api/generate'
data = {
    # "model": "llama3",
    "model": "llama3:instruct",
    # "model": "llama3:8b",
    "prompt": prompt,
    "temperature": 0,
}

In [150]:
headers = {'Content-Type': 'application/json'}
response = requests.post(url, data=json.dumps(data), headers=headers, stream=True)
try:
    count = 0
    for line in response.iter_lines():
        # filter out keep-alive new lines
        # count += 1
        # if count % 5== 0:
        #     print(decoded_line['response']) # print every fifth token
        if line:
            decoded_line = json.loads(line.decode('utf-8'))
            full_response.append(decoded_line['response'])
finally:
    response.close()
print(''.join(full_response))

You're already an outdoor enthusiast! Try exploring new trails and taking in breathtaking views.
